# Load and Validate Raw Data 
This notebook was to  focus on reading the raw data from S3, inspecting the dataset, and validating its contents but 
the platform wasnt available at the time . Therefore we use Local File sysytem

Recommendation:
If scalability is important:

Use Google Cloud Storage (GCS) or Azure Data Lake (ADLS) for a seamless cloud alternative to AWS S3. If working locally or on a small project:
Use the local file system or MinIO for simple and lightweight storage.

In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Customer Churn Data Ingestion") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.6") \
    .getOrCreate()

print("Spark session initialized.")

# Define dataset path (local or S3)
# Change the path to your dataset file or S3 bucket if using AWS
dataset_path = r"C:\Users\ADMIN\Desktop\Projects\Batch-Processing-Project-Customer-Churn-Prediction-Pipeline\datasets\Bank Customer Churn Prediction.csv"

# Load the dataset into Spark DataFrame
try:
    churn_data = spark.read.csv(dataset_path, header=True, inferSchema=True)
    print("Dataset loaded successfully.")
except Exception as e:
    print(f"Error loading dataset: {e}")

# Show the first few rows of the dataset
churn_data.show(5)

# Print the schema to understand the data structure
print("Dataset Schema:")
churn_data.printSchema()

# Count the number of rows and columns
row_count = churn_data.count()
column_count = len(churn_data.columns)
print(f"The dataset contains {row_count} rows and {column_count} columns.")

# Basic Data Exploration
print("Summary Statistics:")
churn_data.describe().show()

# Check for missing values in the dataset
missing_values = churn_data.select(
    *[col(c).isNull().alias(c) for c in churn_data.columns]
).groupBy().sum()

print("Missing Values per Column:")
missing_values.show()

# Stop the Spark session
spark.stop()
